In [1]:
import numpy as np
import cv2
import pandas as pd
import os
import pickle
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split

In [2]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/test_opencv/'
os.listdir(path)

Mounted at /content/drive


['res10_300x300_ssd_iter_140000_fp16.caffemodel',
 'deploy.prototxt.txt',
 'dlib_face_recognition_resnet_model_v1.dat',
 'mmod_human_face_detector.dat',
 'shape_predictor_5_face_landmarks.dat',
 'shape_predictor_68_face_landmarks.dat',
 'models',
 'faceimage',
 'data_face_features.pickle']

In [3]:
data = pickle.load(open("/content/drive/MyDrive/test_opencv/data_face_features.pickle", mode="rb"))

In [4]:
x = np.array(data['data'])
y = np.array(data['label'])

print(x)
print(y)

[[[ 0.1878655  -0.00216637 -0.01756698 ... -0.02468519  0.00210316
   -0.07394283]]

 [[ 0.05528343  0.1389198   0.17492583 ...  0.03997438  0.03386393
   -0.03481558]]

 [[ 0.1266313   0.13926576  0.10115616 ...  0.0002326   0.01435457
   -0.08064225]]

 ...

 [[ 0.0744021   0.09258966 -0.1253442  ...  0.12670155 -0.01547207
   -0.00762508]]

 [[ 0.01553861  0.22916293  0.00462629 ... -0.08098242  0.02878358
    0.03513504]]

 [[ 0.03610452  0.20234719 -0.02446474 ... -0.06291381  0.03664535
    0.01666091]]]
['1721031324_LeVanTrong' '1721031324_LeVanTrong' '1721031324_LeVanTrong'
 ... '1721030491_Nguyễn Thế Nhật' '1721030491_Nguyễn Thế Nhật'
 '1721030491_Nguyễn Thế Nhật']


In [5]:
x.shape

(4565, 1, 128)

In [6]:
y.shape

(4565,)

In [7]:
x = x.reshape(-1, 128)

In [8]:
x.shape

(4565, 128)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)

In [11]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3652, 128), (913, 128), (3652,), (913,))

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [13]:
def get_report(model, x_train, y_train, x_test, y_test):
  y_pred_train = model.predict(x_train)
  y_pred_test = model.predict(x_test)

  # accuracy score
  acc_train = accuracy_score(y_train, y_pred_train)
  acc_test = accuracy_score(y_test, y_pred_test)

  # f1 score
  f1_score_train = f1_score(y_train, y_pred_train, average="macro")
  f1_score_test = f1_score(y_test, y_pred_test, average="macro")

  # print
  print('Accurency Train = %0.2f'%acc_train)
  print('Accurency Test = %0.2f'%acc_test)
  print('F1 Score Train = %0.2f'%f1_score_train)
  print('F1 Score Test = %0.2f'%f1_score_test)

In [14]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train)

LogisticRegression()

In [15]:
get_report(model_logistic, x_train, y_train, x_test, y_test)

Accurency Train = 0.71
Accurency Test = 0.67
F1 Score Train = 0.49
F1 Score Test = 0.48


In [16]:
model_svc = SVC(probability=True)
model_svc.fit(x_train, y_train)

SVC(probability=True)

In [17]:
get_report(model_svc, x_train, y_train, x_test, y_test)

Accurency Train = 0.91
Accurency Test = 0.82
F1 Score Train = 0.83
F1 Score Test = 0.74


In [18]:
model_rf = RandomForestClassifier(n_estimators=20,)
model_rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=20)

In [19]:
get_report(model_rf, x_train, y_train, x_test, y_test)

Accurency Train = 1.00
Accurency Test = 0.76
F1 Score Train = 1.00
F1 Score Test = 0.65


In [20]:
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('svm', SVC(probability=True)),
    ('rf', RandomForestClassifier(n_estimators=20,)),
], voting='soft', weights=[2,3,1])

In [21]:
model_voting.fit(x_train, y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(n_estimators=20))],
                 voting='soft', weights=[2, 3, 1])

In [22]:
get_report(model_voting, x_train, y_train, x_test, y_test)

Accurency Train = 0.98
Accurency Test = 0.82
F1 Score Train = 0.97
F1 Score Test = 0.75


In [23]:
from sklearn.model_selection import GridSearchCV

model_grid = GridSearchCV(model_voting, param_grid={
    'svm__C': [3,5,7,10],
    'svm__gamma': [0.1, 0.3, 0.5],
    'rf__n_estimators': [5, 10,20],
    'rf__max_depth': [3, 5, 7],
    'voting': ['soft', 'hard']
}, scoring='accuracy', cv=3, n_jobs=1, verbose=2)

In [24]:
model_grid.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   7.8s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   6.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   7.1s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   6.4s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.2s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   5.6s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   7.5s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gam

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=3,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('svm',
                                                     SVC(probability=True)),
                                                    ('rf',
                                                     RandomForestClassifier(n_estimators=20))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'rf__max_depth': [3, 5, 7],
                         'rf__n_estimators': [5, 10, 20],
                         'svm__C': [3, 5, 7, 10], 'svm__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [27]:
model_best_estimator = model_grid.best_estimator_

In [28]:
model_grid.best_score_

0.8420067561398704

In [29]:
pickle.dump(model_best_estimator, open("/content/drive/MyDrive/test_opencv/ml_face_person_identity.pkl", mode='wb'))